This Notebook launches a test cases for the 1D Shock-Acoustic-Wave interaction problem as presented presented in the phd thesis "Implicit Discontinuous Galerkin Shock Tracking methods for Compressible Flows with Shocks" (Vandergrift 2024).

In [1]:
#r "BoSSSpad.dll"
#r "XESTSF.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using ApplicationWithIDT;
using XESTSF;
Init();


In [2]:
BoSSSshell.WorkflowMgm.Init("XESTSF_ShockAcousticInteraction1D");
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.DefaultDatabase


{ Session Count = 15; Grid Count = 15; Path = \\dc3\userspace\sebastian\cluster\XESTSF_ShockAcousticInteraction1D }

In [3]:
BoSSSshell.GetDefaultQueue()

RuntimeLocation,win\amd64
AdditionalEnvironmentVars,"[ [OMP_PROC_BIND, spread] ]"
DeploymentBaseDirectory,\\dc3\userspace\sebastian\cluster\binaries
DeployRuntime,True
Name,Default
DotnetRuntime,dotnet
Username,FDY\sebastian
NumOfAdditionalServiceCores,0
NumOfAdditionalServiceCoresMPISerial,0
NumOfServiceCoresPerMPIprocess,0
ServerName,DC3


In [4]:
string dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path; 
dbPath

\\dc3\userspace\sebastian\cluster\XESTSF_ShockAcousticInteraction1D

all configs to run

In [5]:
int[] Configs = new int[] {1,2,3};
int[] DGdegrees = new int[] {3};
int[] Grid_Res = new int[] {1};
double[] Amplitudes = new double[] {1e-5};
string[] WaveForms= new string[] {"1sinus"};
string[] LevelSetConfigs = new string[] {"true"};//,"false"};
double[] MachLs = new double[] {1.5};
double[] WLs = new double[] {0.8};

run the simulations

In [6]:
//foreach(double iProb in iProbs) {
    double amp=1e-5;
    double amp_neg=0.0;
    double amp_pos=amp;
    double endTime=1.5;
    double shockPos=1.5;
    double wavePos=0.0;
    double scaling=1;
    double waveLength=0.4;
    double MachL=1.50;
                foreach(double waveLength in WLs ){
            foreach(double MachL in MachLs ){
        foreach(double amp in Amplitudes ){
foreach(double config in Configs) {
foreach(string waveform in WaveForms) {
    foreach(string withLS in LevelSetConfigs){
        amp_neg= 0.0;            amp_pos= amp;
         if(config==1){ //downstream negative acoustic wave (slow)
            amp_neg= amp;            amp_pos= 0.0;            endTime=4.0;          shockPos=0.5;            wavePos=0.9;
        }else if(config==2){ //upstream positiv acoustic wave (fast)
            amp_neg= 0.0;            amp_pos= amp;            endTime=1.5;            shockPos=1.5;            wavePos=0.0;
        }else if(config==3){ //upstream negative acoustic wave (still fast)
            amp_neg= amp;            amp_pos= 0.0;            endTime=4.0;           shockPos=1.5;            wavePos=0.3;
        }
    
    foreach(int p in DGdegrees) {
        foreach(int n in Grid_Res) {
            int cells=0;
            bool wLS;
            if(withLS == "true"){
                wLS=true;
                cells = (n+1)*30+1;
            }else{
                wLS=false;
                cells=(n+1)*30;
            }
                var tmp_cntrl = XESTSF.XESTSFHardCodedControl.AcousticWave1D(waveLength: waveLength ,scaling: scaling ,endTime: endTime ,MachL: MachL ,withLevelSet: wLS ,wavePosition: wavePos , shockPosition: shockPos ,p_amp_neg:  amp_neg , p_amp_pos: amp_pos ,waveform: waveform,dgDegree: p ,numOfCellsX: cells ,PlotInterval:-1,numOfCellsT: cells ,dbPath: dbPath );
                Job j = new Job(tmp_cntrl.SessionName, typeof(XESTSF.XESTSFMain));// job name has to be equal to sessionname! for SessionInfoJobCorrelation to work!
                j.RetryCount = 2;
                j.MySetCommandLineArguments("--control", $"cs:XESTSF.XESTSFHardCodedControl.AcousticWave1D(waveLength:{waveLength},scaling:{scaling},endTime:{endTime},MachL:{MachL},withLevelSet:{withLS},wavePosition:{wavePos}, shockPosition:{shockPos},p_amp_neg: {amp_neg}, p_amp_pos:{amp_pos},waveform:\"{waveform}\",dgDegree:{p},numOfCellsX:{cells},PlotInterval:-1,numOfCellsT:{cells},dbPath:@\"{dbPath}\")");
                j.NumberOfThreads=1;
                Console.WriteLine("!!!!!!!!!!!!!!!! WARNING: OPENMP Paralelization turned off !!!!!!!!!!!!");
                j.Activate(BoSSSshell.GetDefaultQueue()); 
                
            
        };                                                                                                                                                          
    };
};
    
}
}
        }
            }
                }



In [ ]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(7*86400 , 60); // wait for one week, or until all jobs finish

## Check if residual norms converged

Load sessions

In [13]:
var sessions = BoSSSshell.WorkflowMgm.DefaultDatabase.Sessions;
sessions

#0: XESTSF_AcousticWave1D	AW_p3_xCells91_tCells91_tMax4_1sinus_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8	5/6/2024 9:31:36 AM	78ffe78d...
#1: XESTSF_AcousticWave1D	AW_p3_xCells91_tCells91_tMax1.5_1sinus_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.8	5/6/2024 9:31:24 AM	e4f6c055...
#2: XESTSF_AcousticWave1D	AW_p3_xCells91_tCells91_tMax4_1sinus_Mach1.5_sP0.5_wP0.9_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8*	5/6/2024 9:31:04 AM	917d7dad...
#3: XESTSF_AcousticWave1D	AW_p3_xCells91_tCells91_tMax4_1sinus_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8	4/10/2024 2:48:59 PM	39e1f7d3...
#4: XESTSF_AcousticWave1D	AW_p3_xCells91_tCells91_tMax4_bump_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8	4/10/2024 2:48:23 PM	73ae7ed9...
#5: XESTSF_AcousticWave1D	AW_p3_xCells91_tCells91_tMax1.5_bump_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.8	4/10/2024 2:47:07 PM	d44f94cb...
#6: XESTSF_AcousticWave1D	AW_p3_xCells91_tCells91_tMax1.5_1sinus_Mach1

Obtain the residual norms

In [18]:
var ass= new MatrixAssembler(1);
var res = sessions.Select(si => ((IDTTimeStepInfo) ((TimestepProxy)si.Timesteps.Last()).GetInternal()).ResHistory.Last());
res

[ 8.047201205560531E-11, 1.6398503260686175E-10, 1.5722681544382512E-12, 8.047201205560531E-11, 3.097451725242989E-11, 1.9692228507243044E-10, 1.6398503260686175E-10, 1.4871063959431757E-10, 7.814338588393121E-11, 1.7249810536058186E-12, 2.2027094638436056E-12, 2.2894582973894015E-12, 1.9680356291241526E-12, 1.5722681544382512E-12, 1.5775143985204186E-12 ]

In [17]:
var ass= new MatrixAssembler(1);
var enres = sessions.Select(si => ((IDTTimeStepInfo) ((TimestepProxy)si.Timesteps.Last()).GetInternal()).EnResHistory.Last());
enres

[ 2.907085902784185E-06, 1.745809546847069E-05, 7.4543819520159555E-06, 2.907085902784185E-06, 1.4130240126653708E-06, 8.214796915593675E-06, 1.745809546847069E-05, 3.729899296497381E-06, 1.8905981149569214E-06, 1.4872496382227096E-05, 1.434238241678581E-05, 7.998861483081464E-06, 2.5208634895048603E-05, 7.4543819520159555E-06, 4.101430917713845E-06 ]

Assert if smaller than threshold

In [23]:
res.ForEach(r => NUnit.Framework.Assert.IsTrue(r<1e-09));
enres.ForEach(r => NUnit.Framework.Assert.IsTrue(r<1e-04));